In [ ]:
import math


def calculate_root_locus_gains(info, typename):

    sum_u = info['PC'][:-1].sum()
    sum_y = info['MEAN_BY_PC'][1:].sum()

    mu = sum_u / (len(info) - 1)
    my = sum_y / (len(info) - 1)

    s1 = 0.0
    s2 = 0.0
    s3 = 0.0
    s4 = 0.0
    s5 = 0.0

    u_line = info['PC'] - mu
    y_line = info['MEAN_BY_PC'] - my

    for i in range(len(info) - 1):
        s1 += y_line[i] ** 2.0
        s2 += u_line[i] * y_line[i]
        s3 += u_line[i] ** 2.0
        s4 += y_line[i] * y_line[i + 1]
        s5 += u_line[i] * y_line[i + 1]

    a = (s3 * s4 - s2 * s5) / (s1 * s3 - s2 ** 2.0)
    b = (s1 * s5 - s2 * s4) / (s1 * s3 - s2 ** 2.0)

    kp = 0.0
    ki = 0.0
    kd = 0.0

    if typename == 'BasicP':
        kp = (1 + a) / b
        ki = 0.0
        kd = 0.0
    elif typename == 'BasicPi':
        kp = (a - 0.36) / b
        ki = (a - b * kp) / b
        kd = 0.0
    elif typename == 'BasicPid':
        kd = 0.11 / b
        kp = (-0.063 + a - 2 * b * kd) / b
        ki = (0.3 - b * kp - b * kd + a) / b

    # Create a dict with the gains
    response = {
        'kp': kp,
        'ki': ki,
        'kd': kd
    }

    return response


def calculate_ziegler_gains(info, typename):
    sum_rate1 = info.loc[info['PC'] == 1, 'ARRIVAL_RATE'].sum()
    sum_rate2 = info.loc[info['PC'] == 2, 'ARRIVAL_RATE'].sum()

    dataSize = (len(info) - 2) / 2.0
    mean_rate1 = sum_rate1 / dataSize
    mean_rate2 = sum_rate2 / dataSize

    diff_rate = mean_rate2 - mean_rate1
    K = diff_rate

    if typename == 'BasicP':
        kp = tau / K * L
        ki = 0.0
        kd = 0.0
    elif typename == 'BasicPi':
        ti = L / 0.3
        kp = 0.9 * tau / K * L
        ki = kp / ti
        kd = 0.0
    elif typename == 'BasicPid':
        ti = 2 * L
        td = 0.5 * L
        kp = 1.2 * tau / K * L
        ki = kp / ti
        kd = kp * td

    response = {
        'kp': kp,
        'ki': ki,
        'kd': kd
    }

    return response


def calculate_amigo_gains(info, typename):
    sum_rate1 = info.loc[info['PC'] == 1, 'ARRIVAL_RATE'].sum()
    sum_rate2 = info.loc[info['PC'] == 2, 'ARRIVAL_RATE'].sum()

    dataSize = (len(info) - 2) / 2.0
    mean_rate1 = sum_rate1 / dataSize
    mean_rate2 = sum_rate2 / dataSize

    diff_rate = mean_rate2 - mean_rate1
    K = diff_rate

    if typename == 'BasicPi':
        ti = (0.35 + 13 * (T ** 2) / ((T ** 2) +
              12 * tau * T + 7 * (tau ** 2))) * tau
        kp = 1 / K * (0.15 + (0.35 - tau * T / ((tau + T) ** 2)) * T / tau)
        ki = kp / ti
        kd = 0.0
    elif typename == 'BasicPid':
        ti = ((0.4 * tau + 0.8 * T) / (tau + 0.1 * T)) * tau
        td = (0.5 * T / (0.3 * tau + T)) * tau
        kp = 1 / K * (0.2 + 0.45 * T / tau)
        ki = kp / ti
        kd = kp * td

    response = {
        'kp': kp,
        'ki': ki,
        'kd': kd
    }

    return response


def calculate_cohen_gains(info, typename):
    sum_rate1 = info.loc[info['PC'] == 1, 'ARRIVAL_RATE'].sum()
    sum_rate2 = info.loc[info['PC'] == 2, 'ARRIVAL_RATE'].sum()

    dataSize = (len(info) - 2) / 2.0
    mean_rate1 = sum_rate1 / dataSize
    mean_rate2 = sum_rate2 / dataSize

    diff_rate = mean_rate2 - mean_rate1
    K = diff_rate
    theta = tau / (tau + T)

    if typename == 'BasicP':
        kp = 1 / K * (1 + (0.35 * theta / (1 - theta))) * T / tau
        ki = 0.0
        kd = 0.0
    elif typename == 'BasicPi':
        ti = ((3.3 - 3.0 * theta) / (1 + 1.2 * theta)) * tau
        kp = 0.9 / K * (1 + (0.92 * theta / (1 - theta))) * T / tau
        ki = kp / ti
        kd = 0.0
    elif typename == 'BasicPid':
        ti = ((2.5 - 2.0 * theta) / (1 - 0.39 * theta)) * tau
        td = ((0.37 * (1 - theta)) / (1 - 0.81 * theta)) * tau
        kp = 1.35 / K * (1 + (0.18 * theta / (1 - theta))) * T / tau
        ki = kp / ti
        kd = kp * td

    response = {
        'kp': kp,
        'ki': ki,
        'kd': kd
    }

    return response


In [ ]:
import pandas as pd

In [ ]:
rl_data = pd.read_csv('./data/root_locus_data.csv')
ot_data = pd.read_csv('./data/others_tunning_methods_data.csv')

In [ ]:
rl_gains_p = calculate_root_locus_gains(rl_data, 'BasicP')
rl_gains_pi = calculate_root_locus_gains(rl_data, 'BasicPi')
rl_gains_pid = calculate_root_locus_gains(rl_data, 'BasicPid')

z_gains_p = calculate_ziegler_gains(ot_data, 'BasicP')
z_gains_pi = calculate_ziegler_gains(ot_data, 'BasicPi')
z_gains_pid = calculate_ziegler_gains(ot_data, 'BasicPid')

a_gains_pi = calculate_amigo_gains(ot_data, 'BasicPi')
a_gains_pid = calculate_amigo_gains(ot_data, 'BasicPid')

c_gains_p = calculate_cohen_gains(ot_data, 'BasicP')
c_gains_pi = calculate_cohen_gains(ot_data, 'BasicPi')
c_gains_pid = calculate_cohen_gains(ot_data, 'BasicPid')

In [ ]:
print('Root Locus Gains')
print('P: ', rl_gains_p)
print('PI: ', rl_gains_pi)
print('PID: ', rl_gains_pid)

print('Ziegler-Nichols Gains')
print('P: ', z_gains_p)
print('PI: ', z_gains_pi)
print('PID: ', z_gains_pid)

print('AMIGO Gains')
print('PI: ', a_gains_pi)
print('PID: ', a_gains_pid)

print('Cohen-Coon Gains')
print('P', c_gains_p)
print('PI', c_gains_pi)
print('PID', c_gains_pid)